In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

import glob
import random
import tensorflow as tf
from shutil import rmtree
from keras.models import load_model
from os.path import join, getctime, basename
from models import *
from utils import *
from data_preprocess import load_prep_data

%load_ext autoreload
%autoreload 2

aug_data = load_prep_data(time=500, debug=True, onehot=True)
x_train = aug_data['x_train']
y_train = aug_data['y_train']
x_valid = aug_data['x_valid']
y_valid = aug_data['y_valid']
x_test = aug_data['x_test']
y_test = aug_data['y_test']

Shape of x_train set: (7191, 22, 250)
Shape of y_train labels: (7191, 4)
Shape of x_valid set: (1269, 22, 250)
Shape of y_valid labels: (1269, 4)
Shape of x_test set: (1772, 22, 250)
Shape of y_test labels: (1772, 4)
Shape of x_train_valid set: (8460, 22, 250)
Shape of y_train_valid labels: (8460, 4)


In [2]:
input_shape = x_train.shape
x_train = tf.transpose( tf.expand_dims(x_train, axis=-1), perm=[0, 2, 3, 1])
x_valid = tf.transpose( tf.expand_dims(x_valid, axis=-1), perm=[0, 2, 3, 1])

print("x_train.shape: {}", format(x_train.shape))

x_train.shape: {} (7191, 250, 1, 22)


2024-03-15 01:19:23.560856: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-15 01:19:23.560999: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-15 01:19:23.561020: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-15 01:19:23.561080: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-15 01:19:23.561162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
config = {
    # Network
    'num_inputs': x_train.shape[0],
    'input_shape': (input_shape[2],1, input_shape[1]),
    'epochs': 30,
    'batch_size': 32, 
    'lr': 0.0001
}
ShallowConvNet = ShallowConvNet()
ShallowConvNet.build_model(config)
history = ShallowConvNet.train(x_train, y_train, x_valid, y_valid, config, get_workpath('ShallowConvNet'))

/Users/peiyuanlee/miniforge3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 250, 1, 40)     │        88,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 250, 1, 40)     │       360,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 250, 1, 40)     │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 250, 1, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 31, 1, 40)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 31, 1, 40)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 1, 40)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1240)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         4,964 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 453,204 (1.73 MB)

 Trainable params: 453,124 (1.73 MB)

 Non-trainable params: 80 (320.00 B)

Model compiled.
Epoch 1/30


2024-03-15 01:19:25.431429: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


113/113 ━━━━━━━━━━━━━━━━━━━━ 22s 174ms/step - accuracy: 0.2829 - loss: 1.7312 - val_accuracy: 0.4137 - val_loss: 1.2693
Epoch 2/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 21s 182ms/step - accuracy: 0.3837 - loss: 1.4318 - val_accuracy: 0.5303 - val_loss: 1.1089
Epoch 3/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.4452 - loss: 1.2967 - val_accuracy: 0.5729 - val_loss: 1.0462
Epoch 4/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.4943 - loss: 1.2038 - val_accuracy: 0.5839 - val_loss: 1.0013
Epoch 5/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.5313 - loss: 1.1375 - val_accuracy: 0.6675 - val_loss: 0.8741
Epoch 6/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 170ms/step - accuracy: 0.5863 - loss: 1.0229 - val_accuracy: 0.6879 - val_loss: 0.8160
Epoch 7/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 164ms/step - accuracy: 0.5945 - loss: 0.9772 - val_accuracy: 0.7384 - val_loss: 0.7599
Epoch 8/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 168ms/step - accuracy: 0.6332 - loss: 0.9109 - val

In [4]:
x_test_transposed = tf.transpose(tf.expand_dims(x_test, axis=-1), perm=[0, 2, 3, 1])
raw = ShallowConvNet.evaluate(x_test_transposed, y_test)
print("Raw Acc result: {}".format(raw[1]))

56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6580 - loss: 0.8420
Raw Acc result: 0.6794582605361938
